## IO编程
https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001431917590955542f9ac5f5c1479faf787ff2b028ab47000

IO在计算机中指Input/Output，也就是输入和输出。由于程序和运行时数据是在内存中驻留，由CPU这个超快的计算核心来执行，涉及到数据交换的地方，通常是磁盘、网络等，就需要IO接口。

比如你打开浏览器，访问新浪首页，浏览器这个程序就需要通过网络IO获取新浪的网页。浏览器首先会发送数据给新浪服务器，告诉它我想要首页的HTML，这个动作是往外发数据，叫Output，随后新浪服务器把网页发过来，这个动作是从外面接收数据，叫Input。所以，通常，程序完成IO操作会有Input和Output两个数据流。当然也有只用一个的情况，比如，从磁盘读取文件到内存，就只有Input操作，反过来，把数据写到磁盘文件里，就只是一个Output操作。

IO编程中，Stream（流）是一个很重要的概念，可以把流想象成一个水管，数据就是水管里的水，但是只能单向流动。Input Stream就是数据从外面（磁盘、网络）流进内存，Output Stream就是数据从内存流到外面去。对于浏览网页来说，浏览器和新浪服务器之间至少需要建立两根水管，才可以既能发数据，又能收数据。

由于CPU和内存的速度远远高于外设的速度，所以，在IO编程中，就存在速度严重不匹配的问题。举个例子来说，比如要把100M的数据写入磁盘，CPU输出100M的数据只需要0.01秒，可是磁盘要接收这100M数据可能需要10秒，怎么办呢？有两种办法：

第一种是CPU等着，也就是程序暂停执行后续代码，等100M的数据在10秒后写入磁盘，再接着往下执行，这种模式称为同步IO；

另一种方法是CPU不等待，只是告诉磁盘，“您老慢慢写，不着急，我接着干别的事去了”，于是，后续代码可以立刻接着执行，这种模式称为异步IO。

同步和异步的区别就在于是否等待IO执行的结果。好比你去麦当劳点餐，你说“来个汉堡”，服务员告诉你，对不起，汉堡要现做，需要等5分钟，于是你站在收银台前面等了5分钟，拿到汉堡再去逛商场，这是同步IO。

你说“来个汉堡”，服务员告诉你，汉堡需要等5分钟，你可以先去逛商场，等做好了，我们再通知你，这样你可以立刻去干别的事情（逛商场），这是异步IO。

很明显，使用异步IO来编写程序性能会远远高于同步IO，但是异步IO的缺点是编程模型复杂。想想看，你得知道什么时候通知你“汉堡做好了”，而通知你的方法也各不相同。如果是服务员跑过来找到你，这是回调模式，如果服务员发短信通知你，你就得不停地检查手机，这是轮询模式。总之，异步IO的复杂度远远高于同步IO。

## 异步IO

在IO编程一节中，我们已经知道，CPU的速度远远快于磁盘、网络等IO。在一个线程中，CPU执行代码的速度极快，然而，一旦遇到IO操作，如读写文件、发送网络数据时，就需要等待IO操作完成，才能继续进行下一步操作。这种情况称为同步IO。

在IO操作的过程中，当前线程被挂起，而其他需要CPU执行的代码就无法被当前线程执行了。

因为一个IO操作就阻塞了当前线程，导致其他代码无法执行，所以我们必须使用多线程或者多进程来并发执行代码，为多个用户服务。每个用户都会分配一个线程，如果遇到IO导致线程被挂起，其他用户的线程不受影响。

多线程和多进程的模型虽然解决了并发问题，但是系统不能无上限地增加线程。由于系统切换线程的开销也很大，所以，一旦线程数量过多，CPU的时间就花在线程切换上了，真正运行代码的时间就少了，结果导致性能严重下降。

由于我们要解决的问题是CPU高速执行能力和IO设备的龟速严重不匹配，多线程和多进程只是解决这一问题的一种方法。

另一种解决IO问题的方法是异步IO。当代码需要执行一个耗时的IO操作时，它只发出IO指令，并不等待IO结果，然后就去执行其他代码了。一段时间后，当IO返回结果时，再通知CPU进行处理。

可以想象如果按普通顺序写出的代码实际上是没法完成异步IO的：

```
do_some_code()
f = open('/path/to/file', 'r')
r = f.read() # <== 线程停在此处等待IO操作结果
# IO操作完成后线程才能继续执行:
do_some_code(r)
```

所以，同步IO模型的代码是无法实现异步IO模型的。

异步IO模型需要一个消息循环，在消息循环中，主线程不断地重复“读取消息-处理消息”这一过程：

```
loop = get_event_loop()
while True:
    event = loop.get_event()
    process_event(event)
```
消息模型其实早在应用在桌面应用程序中了。一个GUI程序的主线程就负责不停地读取消息并处理消息。所有的键盘、鼠标等消息都被发送到GUI程序的消息队列中，然后由GUI程序的主线程处理。

由于GUI线程处理键盘、鼠标等消息的速度非常快，所以用户感觉不到延迟。某些时候，GUI线程在一个消息处理的过程中遇到问题导致一次消息处理时间过长，此时，用户会感觉到整个GUI程序停止响应了，敲键盘、点鼠标都没有反应。这种情况说明在消息模型中，处理一个消息必须非常迅速，否则，主线程将无法及时处理消息队列中的其他消息，导致程序看上去停止响应。

消息模型是如何解决同步IO必须等待IO操作这一问题的呢？当遇到IO操作时，代码只负责发出IO请求，不等待IO结果，然后直接结束本轮消息处理，进入下一轮消息处理过程。当IO操作完成后，将收到一条“IO完成”的消息，处理该消息时就可以直接获取IO操作结果。

在“发出IO请求”到收到“IO完成”的这段时间里，同步IO模型下，主线程只能挂起，但异步IO模型下，主线程并没有休息，而是在消息循环中继续处理其他消息。这样，在异步IO模型下，一个线程就可以同时处理多个IO请求，并且没有切换线程的操作。对于大多数IO密集型的应用程序，使用异步IO将大大提升系统的多任务处理能力。


## 协程

https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001432090171191d05dae6e129940518d1d6cf6eeaaa969000

在学习异步IO模型前，我们先来了解协程。

- 协程，又称微线程，纤程。英文名Coroutine。

协程的概念很早就提出来了，但直到最近几年才在某些语言（如Lua）中得到广泛应用。

- 子程序，或者称为函数，在所有语言中都是层级调用，比如A调用B，B在执行过程中又调用了C，C执行完毕返回，B执行完毕返回，最后是A执行完毕。

所以子程序调用是通过栈实现的，一个线程就是执行一个子程序。

子程序调用总是一个入口，一次返回，调用顺序是明确的。而协程的调用和子程序不同。

- 协程看上去也是子程序，但执行过程中，在子程序内部可中断，然后转而执行别的子程序，在适当的时候再返回来接着执行。

注意，在一个子程序中中断，去执行其他子程序，不是函数调用，有点类似CPU的中断。比如子程序A、B：

```
def A():
    print('1')
    print('2')
    print('3')

def B():
    print('x')
    print('y')
    print('z')
```
假设由协程执行，在执行A的过程中，可以随时中断，去执行B，B也可能在执行过程中中断再去执行A，结果可能是：

```
1
2
x
y
3
z
```

但是在A中是没有调用B的，所以协程的调用比函数调用理解起来要难一些。

看起来A、B的执行有点像多线程，但协程的特点在于是一个线程执行，那和多线程比，协程有何优势？

最大的优势就是协程极高的执行效率。因为子程序切换不是线程切换，而是由程序自身控制，因此，没有线程切换的开销，和多线程比，线程数量越多，协程的性能优势就越明显。

第二大优势就是不需要多线程的锁机制，因为只有一个线程，也不存在同时写变量冲突，在协程中控制共享资源不加锁，只需要判断状态就好了，所以执行效率比多线程高很多。

因为协程是一个线程执行，那怎么利用多核CPU呢？最简单的方法是多进程+协程，既充分利用多核，又充分发挥协程的高效率，可获得极高的性能。

Python对协程的支持是通过generator实现的。

在generator中，我们不但可以通过for循环来迭代，还可以不断调用next()函数获取由yield语句返回的下一个值。

但是Python的yield不但可以返回一个值，它还可以接收调用者发出的参数。


传统的生产者-消费者模型是一个线程写消息，一个线程取消息，通过锁机制控制队列和等待，但一不小心就可能死锁。如果改用协程，生产者生产消息后，直接通过yield跳转到消费者开始执行，待消费者执行完毕后，切换回生产者继续生产，效率极高：

通过g.send(None)或者next(g)启动生成器函数，并执行到第一个yield语句结束的位置。

In [4]:
def consumer():
    r = ''
    while True:
        n = yield r
        if not n:
            return
        print('[CONSUMER] consuming {}...'.format(n))
        r = '200 OK'
        
def produce(c):
    c.send(None)
    n = 0
    while n<5:
        n = n + 1
        print('[PRODUCE] Producing {}...'.format(n))
        r = c.send(n)
        print('[PRODUCE] Consumer return:{}'.format(r))
    c.close()
    
c = consumer()
produce(c)

[PRODUCE] Producing 1...
[CONSUMER] consuming 1...
[PRODUCE] Consumer return:200 OK
[PRODUCE] Producing 2...
[CONSUMER] consuming 2...
[PRODUCE] Consumer return:200 OK
[PRODUCE] Producing 3...
[CONSUMER] consuming 3...
[PRODUCE] Consumer return:200 OK
[PRODUCE] Producing 4...
[CONSUMER] consuming 4...
[PRODUCE] Consumer return:200 OK
[PRODUCE] Producing 5...
[CONSUMER] consuming 5...
[PRODUCE] Consumer return:200 OK


In [5]:
def consumer():
    r = ''
    while True:
        n = yield r
        if not n:
            return
        print('[CONSUMER] consuming {}...'.format(n))
        r = '200 OK'
        
def produce(c):
    c.send(None)
    n = 0
    while n<5:
        n = n + 1
        print('[PRODUCE] Producing {}...'.format(n))
        r = c.send(n)
        print('[PRODUCE] Consumer return:{}'.format(r))
    #c.close()
    
c = consumer()
produce(c)

[PRODUCE] Producing 1...
[CONSUMER] consuming 1...
[PRODUCE] Consumer return:200 OK
[PRODUCE] Producing 2...
[CONSUMER] consuming 2...
[PRODUCE] Consumer return:200 OK
[PRODUCE] Producing 3...
[CONSUMER] consuming 3...
[PRODUCE] Consumer return:200 OK
[PRODUCE] Producing 4...
[CONSUMER] consuming 4...
[PRODUCE] Consumer return:200 OK
[PRODUCE] Producing 5...
[CONSUMER] consuming 5...
[PRODUCE] Consumer return:200 OK


## asyncio

asyncio是Python 3.4版本引入的标准库，直接内置了对异步IO的支持。

asyncio的编程模型就是一个消息循环。我们从asyncio模块中直接获取一个EventLoop的引用，然后把需要执行的协程扔到EventLoop中执行，就实现了异步IO。

用asyncio实现Hello world代码如下：

In [ ]:
import asyncio

@asyncio.coroutine
def hello():
    print('Hello world!')
    r = 

## 理解Python协程:从yield/send到yield from再到async/await
https://blog.csdn.net/soonfly/article/details/78361819


http://python.jobbole.com/81911/
http://python.jobbole.com/86069/

Python中的协程大概经历了如下三个阶段： 
1. 最初的生成器变形yield/send 
2. 引入@asyncio.coroutine和yield from 
3. 在最近的Python3.5版本中引入async/await关键字

### 生成器变形yield/send
普通函数中如果出现了yield关键字，那么该函数就不再是普通函数，而是一个生成器。

In [6]:
from random import randint

def mygen(a):
    while len(a)> 0:
        c = randint(0,len(a)-1)
        yield a.pop(c)
        
a = ['aa','bb','cc']
c = mygen(a)
print(c)
print(list(c))

<generator object mygen at 0x058A5300>
['cc', 'aa', 'bb']


像上面代码中的c就是一个生成器。生成器就是一种迭代器，可以使用for进行迭代。生成器函数最大的特点是**可以接受外部传入的一个变量，并根据变量内容计算结果后返回。**

这一切都是靠生成器内部的send()函数实现的。

In [22]:
def gen():
    value = 0
    while True:
        receive = yield value
        if receive == 'e':
            break
        value = 'got:{}'.format(receive)
        
g = gen()
print(g.send(None))
print(g.send('hello'))
print(g.send(123456))
print(g.send('e'))

0
got:hello
got:123456


StopIteration: 

上面生成器函数中最关键也是最易理解错的，就是receive=yield value这句,如果对循环体的执行步骤理解错误，就会失之毫厘，差之千里。 

其实receive=yield value包含了3个步骤： 

- 1、向函数外抛出（返回）value 
- 2、暂停(pause)，等待next()或send()恢复 
- 3、赋值receive=MockGetValue() 。 这个MockGetValue()是假想函数，用来接收send()发送进来的值

执行流程： 

- 1、通过g.send(None)或者next(g)启动生成器函数，并执行到第一个yield语句结束的位置。这里是关键，很多人就是在这里搞糊涂的。运行receive=yield value语句时，我们按照开始说的拆开来看，实际程序只执行了1，2两步，程序返回了value值，并暂停(pause)，并没有执行第3步给receive赋值。因此yield value会输出初始值0。这里要特别注意：在启动生成器函数时只能send(None),如果试图输入其它的值都会得到错误提示信息。

- 2、通过g.send('hello')，会传入hello，从上次暂停的位置继续执行，那么就是运行第3步，赋值给receive。然后计算出value的值，并回到while头部，遇到yield value，程序再次执行了1，2两步，程序返回了value值，并暂停(pause)。此时yield value会输出”got: hello”，并等待send()激活。

- 3、通过g.send(123456)，会重复第2步，最后输出结果为”got: 123456″。

- 4、当我们g.send(‘e’)时，程序会执行break然后推出循环，最后整个函数执行完毕，所以会得到StopIteration异常。

从上面可以看出， 在第一次send(None)启动生成器（执行1–>2，通常第一次返回的值没有什么用）之后，对于外部的每一次send()，生成器的实际在循环中的运行顺序是3–>1–>2，也就是先获取值，然后dosomething，然后返回一个值，再暂停等待。

### yield from

In [23]:
def g1():
    yield range(5)
    
def g2():
    yield from range(5)
    
it1 = g1()
it2 = g2()
for x in it1:
    print(x)
    
for x in it2:
    print(x)

range(0, 5)
0
1
2
3
4


- yield就是将range这个可迭代对象直接返回了。 
- yield from解析了range对象，将其中每一个item返回了。

yield from iterable本质上等于for item in iterable: yield item的缩写版 

来看一下例子，假设我们已经编写好一个斐波那契数列函数

In [10]:
def fib(max):
    n,a,b = 0,0,1
    while n < max:
        yield b
        a, b = b, a + b
        n = n + 1
f=fib(5) 
f

<generator object fib at 0x040B9570>

fib不是一个普通函数，而是一个生成器。因此fib(5)并没有执行函数，而是返回一个生成器对象

(生成器一定是迭代器iterator，迭代器一定是可迭代对象iterable) 

现在我们来看一下，假设要在fib()的基础上实现一个函数，调用起始都要记录日志

In [29]:
def f_wrapper(fun_iterable):
    print('start')
    for item in fun_iterable:
        yield item
    print('end')

wrap = f_wrapper(fib(5))
for i in wrap:
    print(i,end='')

start
11235end


现在使用yield from代替for循环

In [11]:
import logging
def f_wrapper2(fun_iterable):
    print('start')
    yield from fun_iterable  #注意此处必须是一个可生成对象
    print('end')

wrap = f_wrapper2(fib(5))
for i in wrap:
    print(i,end='')

start
11235end


再强调一遍：yield from后面必须跟iterable对象(可以是生成器，迭代器)

`yield from iterable`本质上等于`for item in iterable: yield item`的缩写版